In [ ]:
import matplotlib as mpl
%matplotlib notebook
import matplotlib.pyplot as plt
import time
import numpy as np
import pims,os,imageio,glob
import trackpy as tp
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from scipy.interpolate import interp1d
import pandas as pd

In [ ]:
###INTERPOLATION NOW INCLUDED IN get_trackpy_trajectories.py###
#loop through stages
for stage_num in [s for s in range(1,43) if s!=25]:
    
    #load in trajectory
    with open("trajectories/"+str(stage_num)+"_traj_data.p","rb") as f:
        t = pd.read_pickle(f)

    #loop through particles
    for j in t.particle.unique():

        #find particle
        particle = t[t['particle']==j]

        time = np.array(particle['frame'])
        x = np.array(particle['x'])
        y = np.array(particle['y'])


        if len(time) > 1:
            #all the frames between cell's first and last appearance
            all_frames = np.arange(particle['frame'].iloc[0],particle['frame'].iloc[-1]+1)

            #interpolate x,y
            fx = interp1d(time,x)
            x_filled = fx(all_frames)
            fy = interp1d(time,y)
            y_filled = fy(all_frames)

            data = {'x':x_filled,'y':y_filled,'particle':j,'frame':all_frames}

        else:
            #no need to interpolate if only one time frame

            data = {'x':x,'y':y,'particle':j,'frame':time}

        df_new = pd.DataFrame(data)

        if j == 0:
            df = df_new
        else:
            df = pd.concat([df,df_new])
   
    #save
    df.to_pickle("trajectories/"+str(stage_num)+"_traj_data_filled_in.p")